In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


In [ ]:
#import imutils
#import time
#import cv2
#from google.colab.patches import cv2_imshow
#cv2_imshow(imutils.url_to_image(item['src']))
import sys
import os
import pandas as pd
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)


In [ ]:
shop_name = '---x-Shop-Name-x---'

#shop_directory = f'/content/drive/MyDrive/{shop_name}'
shop_directory = f'/content/{shop_name}'
if not os.path.exists(shop_directory):
    os.makedirs(shop_directory)

url = f'https://www.etsy.com/in-en/shop/{shop_name}'

driver.get(url)


In [ ]:
#attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', item.find_element_by_xpath('.//img'))
#print(attrs)


In [ ]:
product_list_url = []
total_pages = int(driver.find_elements_by_xpath('//a[@class="wt-action-group__item wt-btn wt-pr-xs-2 wt-pl-xs-2"]')[-1].get_attribute('data-page'))

for page_no in range(1,total_pages+1):
  driver.get(f'https://www.etsy.com/in-en/shop/{shop_name}?page={page_no}')
  for item in driver.find_elements_by_xpath('//div[@class="responsive-listing-grid wt-grid wt-grid--block justify-content-flex-start wt-mb-xs-3"]//a'):
    product_list_url.append(item.get_attribute('href').split('?')[0])


In [ ]:
print(f'Total number of pages found : {total_pages}')
print(f'Total number of items found : {len(product_list_url)}')


In [ ]:
data_cols = ['Link','ID','Title','Price','Highlights','Description','Order_Place','Order_Dispatch','Delivery_Date','Delivery_Cost']
df = pd.DataFrame(columns=data_cols)
img_res = '794xN' 

for link in product_list_url:
  driver.get(link)

  head = driver.find_element_by_xpath('//h1[@class="wt-text-body-01 wt-text-left-xs wt-line-height-tight wt-break-word wt-text-truncate--multi-line"]')
  dates = driver.find_elements_by_xpath('//p[@class="wt-mt-xs-2 wt-text-black wt-text-caption-title wt-line-height-tight"]') or [driver.find_element_by_xpath('//p[@class="wt-text-body-03 wt-mt-xs-1 wt-line-height-tight"]')]*3

  data_list = [
               link,
               driver.current_url.split('/')[5],
               head.text,
               driver.find_element_by_xpath('//p[@class="wt-text-title-03 wt-mr-xs-2"]').text,
               [info.text for info in driver.find_elements_by_xpath('//ul[@class="wt-text-body-01  wt-pl-xs-0"]//li')],
               driver.find_element_by_xpath('//div[@data-id="description-text"]').text,
               dates[0].text,
               dates[1].text,
               dates[2].text,
               driver.find_element_by_xpath('//div[@data-estimated-shipping]//p').text
  ]

  img_directory = f'{shop_directory}/items/{data_list[1]}/'
  if not os.path.exists(img_directory):
    os.makedirs(img_directory)

    for img_link in driver.find_elements_by_xpath('//ul[@class="wt-list-unstyled wt-overflow-hidden wt-position-relative carousel-pane-list"]//img'):
      src = img_link.get_attribute('src')
      !wget $src -P $img_directory

  df.loc[len(df)] = data_list
  pd.DataFrame([data_list],columns=data_cols).to_csv(f'{img_directory}/{data_list[1]}.csv', encoding='utf-8-sig')
  df.to_csv(f'{shop_directory}/{shop_name}.csv', encoding='utf-8-sig')

  print(f'{len(df)} entries are scraped')
